In [1]:
# LIBRARIES
#libraries
import os
import pandas as pd
from datetime import datetime as dt
from datetime import time as t
from datetime import timedelta
from pandas.tseries.offsets import MonthEnd
from IPython.display import display
import json
import numpy as np
import re
from openpyxl import load_workbook
import xlsxwriter
import requests
from requests.auth import HTTPBasicAuth

In [2]:
# source
user_credential = r'/Users/dinhhoang.nguyen.CONCENTRIX/OneDrive - Concentrix Corporation\WFM-internal'

time_reset = dt(year=2023, month=8, day=1)
update_less = False

# IMPORT MASTER ROSTER
masterroster = pd.read_json(os.path.join(r'C:', user_credential, r'DB\filejson\masterroster.json'))
masterroster['Employee_ID'] = masterroster['Employee_ID'].astype("Int64")

# IMPORT EPS
if update_less is True:
    eps = pd.read_json(os.path.join(r'C:', user_credential, r'DB\filejson\eps.json'))
else:
    eps = pd.read_json(os.path.join(r'C:', user_credential, r'DB\filejson\eps_full.json'))

# IMPORT ATTENDANCE
if update_less is True:
    atd = pd.read_json(os.path.join(r'C:', user_credential, r'DB\filejson\booking_attendance.json'))
else:
    atd = pd.read_json(os.path.join(r'C:', user_credential, r'DB\filejson\booking_attendance_full.json'))

# IMPORT WORKPLAN
workplan = pd.read_json(os.path.join(r'C:', user_credential, r'DB\filejson\workplan_merge_iex.json'))

# IMPORT REQUIREMENT
requirement = pd.read_json(os.path.join(r'C:', user_credential, r'DB\filejson\requirement.json'))

# IMPORT PROJECTED SHRINKAGE
shrinkage = pd.read_json(os.path.join(r'C:', user_credential, r'DB\filejson\ioshrinkage.json'))


In [3]:
# main_frame ref from ATD
main_frame = atd[['Emp ID', 'Date', 'Shift', 'Ramco', 'Supervisor','LOB', 'CUIC Actual hrs', 'Scheduled hrs', 'Total scheduled', 'Attendance', 'OT']]
# Add Dimension to main_frame
main_frame['Week'] = pd.to_datetime(main_frame['Date'], format='mixed').dt.strftime('%Y%W')
main_frame['Week'] = main_frame['Week'].astype('int64')

C:\Users\dinhhoang.nguyen.CONCENTRIX\AppData\Local\Temp\VSCodePortable-x64Temp\ipykernel_14700\2833043742.py:4: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame.
Try using .loc[row_indexer,col_indexer] = value instead

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy
  main_frame['Week'] = pd.to_datetime(main_frame['Date'], format='mixed').dt.strftime('%Y%W')
C:\Users\dinhhoang.nguyen.CONCENTRIX\AppData\Local\Temp\VSCodePortable-x64Temp\ipykernel_14700\2833043742.py:5: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame.
Try using .loc[row_indexer,col_indexer] = value instead

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy
  main_frame['Week'] = main_frame['Week'].astype('int64')


In [4]:
# Process workplan
## Add in IEX Mapping
jeo_workplan = workplan[['EID', 'Date', 'Scheduled Activity', 'Length']]

act_list = ['Break Offline', 'Coaching 1:1', 'Team Meeting', 'Email 1', 'Break', 'Training Offline', 'Lunch', 'Open Time']

for i in act_list:
    jeo_workplan[i] = jeo_workplan.apply(lambda x: x['Length']*24 if x['Scheduled Activity'] == i else 0, axis=1)

jeo_workplan['Others_wp'] = jeo_workplan.apply(lambda x: x['Length']*24 if x['Scheduled Activity'] not in act_list else 0, axis=1)

jeo_workplan = jeo_workplan.drop(columns={'Length', 'Scheduled Activity'}) # type: ignore

act_list.append('Others_wp')
jeo_workplan = jeo_workplan.groupby(['EID', 'Date'], as_index=False)[act_list].sum()

C:\Users\dinhhoang.nguyen.CONCENTRIX\AppData\Local\Temp\VSCodePortable-x64Temp\ipykernel_14700\2738196075.py:8: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame.
Try using .loc[row_indexer,col_indexer] = value instead

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy
  jeo_workplan[i] = jeo_workplan.apply(lambda x: x['Length']*24 if x['Scheduled Activity'] == i else 0, axis=1)
C:\Users\dinhhoang.nguyen.CONCENTRIX\AppData\Local\Temp\VSCodePortable-x64Temp\ipykernel_14700\2738196075.py:8: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame.
Try using .loc[row_indexer,col_indexer] = value instead

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy
  jeo_workplan[i] = jeo_workplan.apply(lambda x: x['Length']*24 if x['Scheduled Activity

In [5]:
# Add workplan to main_frame
main_frame = pd.merge(main_frame, jeo_workplan, left_on = ['Emp ID', 'Date'], right_on=['EID', 'Date'], how='left')
main_frame = main_frame.drop(columns={'EID'})
main_frame[act_list] = main_frame[act_list].fillna(0)
main_frame = main_frame.rename(columns={'Break Offline': 'Break Offline_wp', 'Coaching 1:1': 'Coaching 1:1_wp', 'Team Meeting': 'Team Meeting_wp', 'Email 1': 'Email 1_wp', 'Break': 'Break_wp', 'Training Offline': 'Training Offline_wp', 'Lunch': 'Lunch_wp', 'Open Time': 'Open Time_wp'})

In [6]:
# Get requirement
jeo_requirement = requirement[['LOB', 'Date', 'Daily Requirement', 'Prod Requirement']]
jeo_requirement = jeo_requirement.rename(columns={'Daily Requirement': 'Delivery Requirement', 'Prod Requirement': 'Productive Requirement'})

In [7]:
# Add requirement into main_frame
main_frame = pd.merge(main_frame, jeo_requirement, on=['LOB', 'Date'], how='left')
main_frame[['Delivery Requirement', 'Productive Requirement']] = main_frame[['Delivery Requirement', 'Productive Requirement']].fillna(0)

In [8]:
# EPS processing
jeo_eps = eps
# jeo_eps['Session Login'] = pd.to_datetime(jeo_eps['Session Login'], format='mixed')
# jeo_eps['Session Login'] = jeo_eps['Session Login'] - pd.Timedelta(hours=7)
# jeo_eps['Session Login'] = jeo_eps['Session Login'].dt.tz_localize('UTC', ambiguous=True).dt.tz_convert('Europe/Berlin')
jeo_eps['Session Login'] = pd.to_datetime(jeo_eps['Session Login'], format='mixed').dt.date
bpe_code = ['Picklist - off Phone', 'Ready or Talking', 'Lunch', 'Break', 'Meeting', 'Not Working Yet', 'Training', 'RONA']
for i in bpe_code:
    jeo_eps[i] = jeo_eps.apply(lambda x: x['Total Time']/3600 if x['BPE Code'] == i else 0, axis=1)
jeo_eps['Others_eps'] = jeo_eps.apply(lambda x: x['Total Time']/3600 if x['BPE Code'] not in bpe_code else 0, axis=1)
bpe_code.append('Others_eps')
jeo_eps = jeo_eps.groupby(['EID', 'Session Login'], as_index=False)[bpe_code].sum()

In [9]:
# Add EPS to mainframe
main_frame['Date'] = pd.to_datetime(main_frame['Date'], format='mixed').dt.date
main_frame = pd.merge(main_frame, jeo_eps, left_on=['Emp ID', 'Date'], right_on=['EID', 'Session Login'], how='left')
main_frame = main_frame.drop(columns={'EID', 'Session Login'})
main_frame = main_frame.rename(columns={'Picklist - off Phone': 'Picklist - off Phone_eps', 'Ready or Talking': 'Ready or Talking_eps', 'Lunch': 'Lunch_eps', 'Break': 'Break_eps', 'Meeting': 'Meeting_eps', 'Not Working Yet': 'Not Working Yet_eps', 'Training': 'Training_eps', 'RONA': 'RONA_eps'})

In [10]:
# Add shrinkage
main_frame = pd.merge(main_frame, shrinkage, on = ['LOB', 'Week'], how='left')
main_frame = main_frame.drop(columns={'filename'})

In [11]:
# Export to CSV
# main_frame.to_json((os.path.join(r'C:', user_credential, r'DB\filejson\jeopardy.json')), orient='records')
main_frame.to_csv(r'C:\Users\dinhhoang.nguyen.CONCENTRIX\Concentrix Corporation\Tung Quan Le - BKN\Jeopady\[BcomDB] Jeopardy.csv', index=False)
main_frame.to_csv((os.path.join(r'C:', user_credential, r'DB\filejson\[BcomDB] Jeopardy.csv')), index=False)